# `SelectWhen` Enumerable Extension

## Overview

`.TryGet(out ..)` 혹은 [`pattern-matching`](https://learn.microsoft.com/dotnet/csharp/fundamentals/functional/pattern-matching) 스타일의 코드는 절차적인 코드와는 잘 연계되나 `Linq` 와 같은 함수형 표현과는 상성이 좋지 못한편입니다.

In [13]:
#nullable enable
string?[] numbers = [ "1", "2", "", null, "four" ];
numbers

[ 1, 2, , <null>, four ]

위와같은 임의의 문자열 배열에서 유효한 정수문자열을 추출해 사용한다고 가정해봅시다.

In [16]:
#nullable enable
using System.Collections.Generic;
static IEnumerable<int> TryParseInt(this IEnumerable<string?> source)
{
    foreach (var str in source)
        if (int.TryParse(str, out var i))
            yield return i;
}
numbers.TryParseInt().ToArray()

[ 1, 2 ]

이렇게 하면 절차적으로 자연스럽게 유효한 값만 서브할수 있으나, 람다식에서는 `yield` 문법을 사용할수 없으므로 필요할때마다 반복기 함수를 따로 선언해야만하는 문제가 있습니다.

In [120]:
numbers.Where(str => int.TryParse(str, out _)).Select(str => int.Parse(str)); // [ 1, 2 ]
numbers.Select(str => (b: int.TryParse(str, out var i), i)).Where(tuple => tuple.b).Select(tuple => tuple.i); // [ 1, 2 ]
from str in numbers where int.TryParse(str, out var i) select i

Error: (3,63): error CS0103: 'i' 이름이 현재 컨텍스트에 없습니다.

`where` 에서는 반환값 외의 `out` 출력을 사용할수가 없어 불필요하게 연산을 두번해야합니다.

억지로 사용해보려 엮어도 더 복잡해 보이기만 할 뿐 입니다.

In [30]:
#nullable enable
var arr = new[]
{
    new { Name = "Alice", Age = 30 },
    new { Name = "Bob", Age = -1 },
    null,
    new { Name = "Charlie", Age = 25 },
    new { Name = "David", Age = 0 }
};
arr.OfType<object>()

index value 0 { Name = Alice, Age = 30 } Name Alice Age 30 1 { Name = Bob, Age = -1 } Name Bob Age -1 2 { Name = Charlie, Age = 25 } Name Charlie Age 25 3 { Name = David, Age = 0 } Name David Age 0

null인 값은 적절히 필터링 했으나 정작 익명객체의 맴버에 접근할수가 없습니다.

In [40]:
#nullable enable
arr.Where(a => a is { } notnull).Select(a => a!.Name)

[ Alice, Bob, Charlie, David ]

패턴매칭을 통한 변수캡쳐를 활용못한채 컴파일러가 흐름을 분석해주지 않으므로, null이 아닌값을 필터링 했음에도 불구하고 여전히 null 가능성을 경고하기때문에, `!.` 을 사용해 null이 포함되지 않았음을 억지로 지정해야 합니다.

`Naratteu.Linq` 는 이러한 함수형 표현의 한계를 `Select` + `When` 조합으로 해소하고자 구성되엇습니다.

## Usage

In [45]:
#r "nuget: Naratteu.Linq, 0.0.1"
using static Naratteu.Linq.SelectWhen;

Installed Packages Naratteu.Linq, 0.0.1

In [46]:
numbers.Select(str => When(int.TryParse(str, out var i), out i))

[ 1, 2 ]

In [47]:
arr.Select(a => When(a is { Age: > 0, Name: var name }, out name))

[ Alice, Charlie ]

캡쳐한 변수를 `out` 하는 최소한의 표현만으로 유효한 값을 추출해냅니다.

## Advanced

### [`type` + `const` + `inner capture`](https://dotnetfiddle.net/skR9iN)

### `select` + `switch` + `when` complex use case

In [121]:
using static System.Globalization.NumberStyles;
record IntBox(int Int);
new object[] { 1, "2", 3.0, 3.1, 4f, "0xFF", "0b11", new IntBox(10), null }.Select(o => o switch
{
    int i => When(true, out i),
    string s when s.StartsWith("0x") => When(int.TryParse(s[2..], HexNumber, null, out var i), out i),
    string s and ['0', 'b' or 'B', ..] => When(int.TryParse(s[2..], BinaryNumber, null, out var i), out i),
    string s => When(int.TryParse(s, out var i), out i),
    double d => When((int)d is var i && i == d, out i),
    IConvertible conv => When(conv.ToInt32(null) is var i && conv.ToDecimal(null) == i, out i),
    IntBox(Int: var i) => new(true, i),
    _ => default
})

[ 1, 2, 3, 4, 255, 3, 10 ]

## Todos

- ~~[RefLinq](https://dotnetfiddle.net/ucAXwJ)~~ [`ZLinq`](https://github.com/Cysharp/ZLinq) 지원
    - SelectWhen.Pair<T> 객체가 단순 값복사가 아닌 `ref` 참조를 담아 전달될 필요가 있음.